In [ ]:
import requests
import re 
from bs4 import BeautifulSoup
import pandas as pd

import html5lib
import re
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import warnings

warnings.simplefilter("ignore")

## Web scraping - read html and Beautiful Soup

In [ ]:
#function for scraping director pages on Rotten Tomatoes 

tbls1 = pd.read_html("http://www.rottentomatoes.com/celebrity/wes_anderson")
tbls1 = tbls1[1]
tbls1["Name"] = "Wes Anderson"

In [ ]:
#appending scraped pages and creating df
frames = [tbls1, tbls2, tbls3, tbls4, tbls5, tbls6, tbls7, tbls8, tbls9, tbls10, tbls11, tbls12, tbls13, tbls14, tbls15, tbls16,
         tbls17,tbls18,tbls19,tbls20]

df = pd.concat(frames)
df["MOVIECOUNT"] = 1
cols = ["RATING", "TITLE", "CREDIT", "BOX OFFICE", "YEAR", "NAME", "MOVIECOUNT"]
df.columns = cols


## Munging
Each function in the below cell counts a 0 or a 1 if it detects the needed word in the 'credits' cell from Rotten Tomatoes. These words are always spelled correctly. To handle the actor column, I wanted to be sure not to pick up any cameos (which often include the name 'man on a train...' or 'cameo' or 'himself'. Since speaking roles were what I was after, I cleaned out the rest of the cell contents and if anything was left after trimming the fat, I counted that as a character name. In this situation, it helped to 'know' my data / to have a small enough dataframe to be able to see patterns in cameo names. 

In [ ]:
df['RATING'] = pd.DataFrame.replace(df['RATING'], to_replace= np.nan, value = 0)
df["RATING"] = df["RATING"].map(lambda x: str(x)[:2])
df["RATING"] = df["RATING"].map(lambda x: re.sub('[^0-9]','', x))

df['DIRECTOR'] = 0
df['PRODUCER'] = 0
df['EXECUTIVEPRODUCER'] = 0
df['ACTOR'] = 0
df['WRITER'] = 0
df["CAMEO"] = 0

def director(credit):
    if re.search("Director", credit):
        return(1)
    else:
        return(0)
    
def producer(credit):
    if re.search("Executive", credit):
        return(0)
    elif re.search("Producer", credit):
        return(1)
    else:
        return(0)

def eproducer(credit):
    if re.search("Executive Producer", credit):
        return(1)
    else:
        return(0)

def writer(credit):
    if re.search("Screenwriter", credit):
        return(1)
    else:
        return(0)

def cameo(credit):
    if re.search("Actor", credit):
        return(1)
    elif re.search("Man", credit):
        return(1)
    else:
        return(0)
    
def actor(credit):
    if re.search("Man", credit):
        return(0)
    else: 
        if re.sub("Executive", "", re.sub("Director", "", 
                                      re.sub("Screenwriter", "", 
                                             re.sub("Producer", "", 
                                                    re.sub("Actor", "", 
                                                           re.sub("Himself", "", credit)))))).strip() != "":
            return(1)
        else:
            return(0)

df['DIRECTOR'] = df['CREDIT'].apply(director)
df['ACTOR'] = df['CREDIT'].apply(actor)
df['CAMEO'] = df['CREDIT'].apply(cameo)
df['PRODUCER'] = df['CREDIT'].apply(producer)
df['EXECUTIVEPRODUCER'] = df['CREDIT'].apply(eproducer)
df['WRITER'] = df['CREDIT'].apply(writer)
df["LIT"] = df["EXECUTIVEPRODUCER"] + df["WRITER"] + df["PRODUCER"] + df["ACTOR"]
df["RATING"] = df["RATING"].map(lambda x: int(x))
df = df[df["DIRECTOR"] > 0] #removes movies from director pages where they didn't direct
df = df[df["RATING"] > 0]

In [ ]:
df = pd.to_pickle(df, "directors.pickle") #storing progress

In [ ]:
#this is scraped information from a metis classmate!
p = "df_all.pickle"
janine = pd.read_pickle(p)
cols = ['BUDGET', 'CLOSEDATE', 'DIST', 'DOMESTICGROSS', 'FOREIGNCROSS', 'GENRE','INRELEASE', 'MPAA', 
        'RELEASEDATE', 'RUNTIME', 'TITLE', 'WIDEST', 'LINK']

janine.columns = cols
janine = pd.to_pickle(janine, "df_all.pickle")
janine = "df_all.pickle"
janine = pd.read_pickle(janine)

## Combining dataframes from different sources, pickling for later

In [ ]:
nudf = pd.merge(df, pickle, on="TITLE")
cols = ['RATING', 'TITLE', 'CREDIT', 'BOXOFFICE', 'YEAR', 'NAME', 'MOVIECOUNT',
       'DIRECTOR', 'PRODUCER', 'EXECUTIVEPRODUCER', 'ACTOR', 'WRITER', 'CAMEO',
       'LIT', 'BUDGET', 'CLOSEDATE', 'DIST', 'DOMESTICGROSS', 'FOREIGNCROSS',
       'GENRE', 'INRELEASE', 'MPAA', 'RELEASEDATE', 'RUNTIME', 'WIDEST',
       'LINK']
nudf.columns = cols
nudf["BUDGET"] = pd.DataFrame.replace(nudf["BUDGET"], to_replace= np.nan, value = 0)
nudf["BOXOFFICE"] = nudf["BOXOFFICE"].map(lambda x: re.sub('[^0-9]','', x)) #removing non-integers
nudf["BOXOFFICE"] = pd.DataFrame.replace(nudf["BOXOFFICE"], to_replace= "", value = 0) #handling blanks will be removed
nudf["BOXOFFICE"] = nudf["BOXOFFICE"].map(lambda x: int(x))
nudf.head(5)

In [ ]:
#combining pickles
nudf = nudf[nudf["BOXOFFICE"] > 0]
nudf = pd.to_pickle(nudf, "full_luther.pickle")
nudf = "full_luther.pickle"
nudf = pd.read_pickle(nudf)

## Modeling - OSL
I tried a few different tactics - ultimately I decided to create interactions between the various LIT categories. 

In [ ]:
#another dataframe from a generous classmate
rafa = pd.read_pickle("rtscrapedgood.pkl")
rafa.columns  = ['TITLE', 'YR', 'YR2', 'URL', 'CRITICSCORE',
       'AUDIENCESCORE']
nudf = pd.merge(nudf, rafa, how="inner", on="TITLE")

In [ ]:
nudf["ACTORPROD"] = nudf["ACTOR"] * nudf["PRODUCER"]
nudf["ACTOREP"] = nudf["ACTOR"] * nudf["EXECUTIVEPRODUCER"]
nudf["ACTORWRIT"] = nudf["ACTOR"] * nudf["WRITER"]
nudf["WRITPROD"] = nudf["PRODUCER"] * nudf["WRITER"]
nudf["WRITEEP"] = nudf["EXECUTIVEPRODUCER"] * nudf["WRITER"]
nudf["FULLYOLO_EP"] = nudf["ACTOR"] * nudf["EXECUTIVEPRODUCER"] * nudf["WRITER"]
nudf["FULLYOLO_PROD"] = nudf["ACTOR"] * nudf["PRODUCER"] * nudf["WRITER"]
nudf = pd.to_pickle("absrottentomatos.pickle") #pickling yet again

In [ ]:
nudf.corr() #checking correlations

In [ ]:
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression

lm1 = smf.ols("DOMESTICGROSS ~ WRITEEP + FULLYOLO_EP + FULLYOLO_PROD + BUDGET + ACTORPROD + WRITPROD", data=nudf)

fit1 = lm1.fit()
fit1.summary()

## Cross Validation - Kfold, Lasso
I suspected that I had too few data points for how many features I had. I decided to do a cross validation on my dataframe to see if this was the case. 

In [ ]:
from sklearn.cross_validation import KFold
from sklearn import linear_model

temp = nudf[["WRITEEP", "FULLYOLO_EP", "FULLYOLO_PROD", "BUDGET", "ACTORPROD", "WRITPROD", 'AUDIENCESCORE']]
temp = temp.fillna(value = 0)

x, y = temp.iloc[:,:-1], temp["AUDIENCESCORE"]
x = np.array(x)
y = np.array(y)

kf = KFold(141, n_folds = 5)
rng = np.arange(1, 20, 1)
alphalist = []
SSElist = []
for a in rng:
    SSE = 0
    alphalist.append(a)
    for train_index, test_index in kf:
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        cf = linear_model.Lasso(alpha=a)
        cf.fit(x_train, y_train)
        y_pred = cf.predict(x_test)
        SSE += ((y_pred - y_test) **2).sum() 
    SSE = SSE / 5
    SSElist.append(SSE)

In [ ]:
#plotting the SSE against the lambda. 

import matplotlib.pyplot as plt 

%matplotlib inline
#lambda increases from left to right (high variance on left, high bias on right)
#overfitting is on the right in this graph, underfit is on the left
plt.plot(alphalist, SSElist)
plt.xlabel("Lambda (alpha)")
plt.ylabel("Error")

temp2 = pd.DataFrame(SSElist, alphalist)
cols = ["column"]
temp2.columns = cols
temp2 = temp2.sort_values("column", ascending = True )
temp2

In [ ]:
model_best = linear_model.Lasso(alpha= 4) #optimized lambda and put it in the linear model again. It decimated many of my coeffs
model_best.fit(x_train, y_train)
model_best.coef_ 

## Reflections, next steps

- I need more data
- this is still an interesting subject, and though the observations are small, it is worth continuing to investigate
- I need to try log transormations, especially on the gross dollar graphs. 
